# Dropout 

## Implementation 

In [1]:
import nnfs
import numpy as np
from nnfs.datasets import spiral_data

In [2]:
nnfs.init()

In [3]:
import os
import sys

module = os.path.abspath(os.path.join(".."))
if module not in sys.path:
    sys.path.append(module)

In [4]:
from Chpt_14.Updated_Classes import *

In [5]:
%run Dropout_Class.ipynb

In [6]:
X, y = spiral_data(samples=1000, classes=3)

In [7]:
dense1 = Layer_Dense(2, 512, weight_regularizer_l2=5e-4, bias_regularizer_l2=5e-4)
activation1 = Activation_ReLU()

In [8]:
dropout1 = Layer_Dropout(0.2)

In [9]:
dense2 = Layer_Dense(512, 3)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

In [10]:
from Chpt_10.Optimizers import Optimizer_Adam

In [11]:
optimizer = Optimizer_Adam(learning_rate=0.05, decay=5e-5)

In [12]:
for epoch in range(10001):

    dense1.forward(X)
    activation1.forward(dense1.outputs)
    
    dropout1.forward(activation1.output)
    
    dense2.forward(dropout1.output)
    data_loss = loss_activation.forward(dense2.outputs, y)

    regularization_loss = loss_activation.loss.regularization_loss(
        dense1
    ) + loss_activation.loss.regularization_loss(dense2)

    loss = data_loss + regularization_loss

    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y)

    if not epoch % 100:
        print(
            f"epoch: {epoch}, "
            + f"acc: {accuracy:.3f}, "
            + f"loss: {loss:.3f} ("
            + f"data_loss: {data_loss:.3f}, "
            + f"reg_loss: {regularization_loss:.3f}), "
            + f"lr: {optimizer.current_learning_rate}"
        )

    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    
    dropout1.backward(dense2.dinputs)
    
    activation1.backward(dropout1.dinputs)
    dense1.backward(activation1.dinputs)
    

    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.379, loss: 1.099 (data_loss: 1.099, reg_loss: 0.000), lr: 0.05
epoch: 100, acc: 0.694, loss: 0.762 (data_loss: 0.703, reg_loss: 0.059), lr: 0.04975371909050202
epoch: 200, acc: 0.748, loss: 0.670 (data_loss: 0.596, reg_loss: 0.074), lr: 0.049507401356502806
epoch: 300, acc: 0.771, loss: 0.623 (data_loss: 0.544, reg_loss: 0.078), lr: 0.0492635105177595
epoch: 400, acc: 0.781, loss: 0.633 (data_loss: 0.554, reg_loss: 0.079), lr: 0.04902201088288642
epoch: 500, acc: 0.799, loss: 0.582 (data_loss: 0.504, reg_loss: 0.078), lr: 0.048782867456949125
epoch: 600, acc: 0.811, loss: 0.571 (data_loss: 0.495, reg_loss: 0.076), lr: 0.04854604592455945
epoch: 700, acc: 0.822, loss: 0.559 (data_loss: 0.478, reg_loss: 0.080), lr: 0.048311512633460556
epoch: 800, acc: 0.819, loss: 0.558 (data_loss: 0.484, reg_loss: 0.074), lr: 0.04807923457858551
epoch: 900, acc: 0.810, loss: 0.596 (data_loss: 0.521, reg_loss: 0.075), lr: 0.04784917938657352
epoch: 1000, acc: 0.826, loss: 0.531 (data_lo

## Validation

In [13]:
X_test, y_test = spiral_data(samples=100, classes=3)

In [14]:
dense1.forward(X_test)
activation1.forward(dense1.outputs)
dense2.forward(activation1.output)
loss = loss_activation.forward(dense2.outputs, y_test)

In [15]:
predictions = np.argmax(loss_activation.output, axis=1)
if len(y.shape) == 2:
    y_test = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_test)

In [16]:
print(f'validation, acc: {accuracy:.3f}, loss: {loss:.3f}')

validation, acc: 0.847, loss: 0.436
